In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [21]:
import torch
import torch.nn as nn
from model_helper import MobileNetV2, Inv2d
import wandb
import mysql.connector as connector
from pathlib import Path
import tqdm
import torchvision
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.models import mobilenet_v2
import PIL.Image as Image
import numpy as np
from torchvision.transforms.functional import pil_to_tensor

from data_helper import compute_gradcam, overlay_heatmap

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image


In [3]:
home = os.path.expanduser('~')
os.chdir(home) # b/c we will be using universal paths

host = '127.0.0.1'
user = 'root' # change to your username
password = 'vasya1' # change to your password
database = 'ai_proj_2025' # we should all have this as the db name 

try:
    conn = connector.connect(
        host = host, 
        user = user, 
        password = password, 
        database = database
    )
    print('success')
except connector.Error as err:
    print(err)

success


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


In [5]:
# research_dir = Path(home, 'Desktop', 'Education', 'Spring 2025', 'AI', 'research')
research_dir = Path(home, "ai_research_proj_spring_2025", "research_ai_class_spring_2025") # in lab 409 computer for Agafia
os.chdir(research_dir)

from data_helper import SQLDataset_Informative

os.chdir(home)

In [6]:
from torchvision.transforms import v2

# transforms
transformations = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True), 
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# create train, val, and test sets

data_dir=Path('OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0')

train_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_train=True)
val_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_val=True)
test_set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=data_dir, is_test=True)

In [8]:
#Define the convnet (Eventually replae with mobile 2)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) #specify the size (outer numbers) and amount (middle number) of filters
        self.pool = nn.MaxPool2d(2, 2) #specify pool size first number is size of pool, second is step size
        self.conv2 = nn.Conv2d(16, 8, 3) #new depth is amount of filters in previous conv layer
        self.fc1 = nn.Linear(54*54*8, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 2) #finial fc layer needs 19 outputs because we have 19 layers # ???

    def forward(self, x):

        x = F.relu(self.conv1(x))
     
        x = self.pool(x)
        
       
        x = F.relu(self.conv2(x))

        x = self.pool(x)
        print(x.shape)

        x = x.view(-1, 54*54*8) # flatten

        x = F.relu(self.fc1(x))    #fully connected, relu         
        x = F.relu(self.fc2(x))    
       
        x = self.fc3(x)     #output    
        return x

In [9]:
os.chdir(home)
model = torch.load(Path(research_dir, 'models', 'Basic CNN final run -- lr=0.001')).to(device)
for param in model.parameters():
    param.requires_grad = True
for name, module in model.named_modules():
    print(name, '->', type(module))


 -> <class '__main__.ConvNet'>
conv1 -> <class 'torch.nn.modules.conv.Conv2d'>
pool -> <class 'torch.nn.modules.pooling.MaxPool2d'>
conv2 -> <class 'torch.nn.modules.conv.Conv2d'>
fc1 -> <class 'torch.nn.modules.linear.Linear'>
fc2 -> <class 'torch.nn.modules.linear.Linear'>
fc3 -> <class 'torch.nn.modules.linear.Linear'>


C:\Users\bowdenaa\AppData\Local\Temp\ipykernel_18680\3680321473.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(Path(research_dir, 'models', 'Basic CN

In [10]:
# image = val_set.__getitem__(2)['image'].to(device)
# label = val_set.__getitem__(2)['label'].item()
# label
image_path = r'C:\Users\bowdenaa\OneDrive - Stephen F. Austin State University\CrisisMMD_v2.0\CrisisMMD_v2.0\data_image\hurricane_harvey\13_9_2017\908031796092252162_0.jpg' 
# img = Image.open(image_path).convert('RGB')
# img = pil_to_tensor(img)
# img = transformations(img).unsqueeze(0).to(device) # add batch dimension
# label = 1 # informative

In [11]:
# map = compute_gradcam(model, img, label, conv_layer_name='conv2')
# map

In [28]:
import cv2


# os.chdir(home)


# overlayed = overlay_heatmap(image_path,map, alpha=.25)
# overlayed   

In [34]:
target_layer = 'conv2'
target_layer = model.conv2
gradcam = GradCAM(model, target_layers=[target_layer])



In [41]:

img = cv2.imread(image_path, cv2.IMREAD_COLOR)


img = cv2.resize(img, (224, 224))
img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(img.shape)

target_category = 1 # informative
greyscale = gradcam(input_tensor)
cam = np.uint8(255*greyscale[0,:])
vis = show_cam_on_image(img, cam, use_rgb=True)

(224, 224, 3)
torch.Size([1, 8, 54, 54])


In [44]:
pil_img = Image.fromarray(vis)
pil_img.show()